In [1]:
import numpy as np
import pandas as pd
from ipynb.fs.full.fun_util import *
import warnings
import sys
if not sys.warnoptions:
    warnings.simplefilter("ignore")
np.__version__
pd.__version__

'1.3.5'

In [2]:
## scenario switch
case=3;  # 2 means no trade in goods; 3 means trade in both energy and goods
logit = 0; # 1 means logit estimations of supply elasticity; 0 means fixed elasticities at 0.5

In [3]:
## import BAU values (seven regional scenarios in the order of US, EU, OECD, World, China, OECD plus China)
if case==2:
    df = pd.read_csv("../output/BaselineCarbon_2015_noTradeinGoods.csv",index_col=['region_scenario','regionbase'],header='infer')
elif case==3:
    df = pd.read_csv("../output/BaselineCarbon_2015.csv", index_col=['region_scenario','regionbase'],header='infer')
df['jxbar']=df['CeFH']/(df['CeFH'] + df['CeFF'])
df['jmbar']=df['CeHH']/(df['CeHH'] + df['CeHF'])
df

,,Qe,Qestar,Qeworld,CeHH,CeHF,CeFH,CeFF,Ce,Cestar,Ge,Gestar,Ceworld,Geworld,jxbar,jmbar
region_scenario,regionbase,,,,,,,,,,,,,,,
1,US as Home,4.480045,27.795946,32.275990,4.598403,1.196111,0.421601,26.059874,5.794514,26.481474,5.020004,27.255985,32.275990,32.275990,0.015921,0.793579
2,EU28 as Home,0.935781,31.340202,32.275986,2.950581,1.013552,0.507676,27.804173,3.964133,28.311848,3.458257,28.817724,32.275982,32.275982,0.017932,0.744319
3,OECD37 as Home,8.625495,23.650503,32.276001,11.293668,2.487537,0.910579,17.584217,13.781205,18.494795,12.204247,20.071754,32.276001,32.276001,0.049234,0.819498
4,World as Home,32.175999,0.100000,32.276001,32.146000,0.030000,0.030000,0.070000,32.175999,0.100000,32.175999,0.100000,32.276001,32.276001,0.300000,0.999068
5,China as Home,7.522744,24.753246,32.275990,7.345464,0.632472,1.935382,22.362669,7.977936,24.298052,9.280846,22.995142,32.275990,32.275990,0.079652,0.920722
6,OECD and China as Home,16.148245,16.127758,32.276001,20.103180,1.655961,1.381915,9.134943,21.759140,10.516858,21.485096,10.790904,32.276001,32.276001,0.131400,0.923896
7,US and EU as Home,5.415825,26.860159,32.275986,7.728876,2.029770,0.749387,21.767950,9.758646,22.517336,8.478263,23.797718,32.275982,32.275982,0.033280,0.792003


In [4]:
## choose which regional scenario to run (runs all if not executed)
    #df=df.drop([1,2,4,5,6,7])  

In [5]:
## parameter values
alpha = 0.85;           # labor share parameter in manufacturing
theta = 4;              # scopevec for comparative advantage
sigma = 1;      # elasticity of demand for each individual manufactured good j at Home
sigmastar = 1;  # elasticity of demand for each individual manufactured good j at Foreign
epsilonD = alpha + (1 - alpha) * sigma;  #Home's elasticity of demand for embodied energy
epsilonDstar = alpha + (1 - alpha) * sigmastar;  #Foreign's elasticity of demand for embodied energy
# beta = 2.274853;
# gamma= 0.784877595;
beta=1.892412;
gamma=0.807998928;
epsilonS = 0.5;  #Homes's energy supply elasticity: beta/(1 - beta)
epsilonSstar = 0.5;  #Foreign's energy supply elasticity: betastar/(1 - betastar)

In [6]:
tax_scenario= pd.DataFrame({'tax_sce': ['Baseline','Unilateral','purete','puretc','puretp','EC_hybrid','EP_hybrid','PC_hybrid','EPC_hybrid'], 'Base':[1,0,1,1,1,1,1,1,1]},index=[1, 2, 3, 4, 5, 6, 7, 8, 9])
# use for quick test: tax_scenario= pd.DataFrame({'tax_sce': 'EPC_hybrid', 'Base':1},index=[1])

In [7]:
def cases(tax_scenario, alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS,epsilonSstar, beta, gamma, logit):
    ParaList = (alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS,epsilonSstar, beta, gamma, logit)
    # varphilist = np.arange (1.6,2.5,0.1) # marginal damages
    varphilist = np.arange (0,20.1,0.2) # marginal damages

    # use for quick test: varphilist = [2] or varphilist = np.arange (1.7,2.5,0.1)
    varphilist = [2]
    output=[]
    for varphi in varphilist:    
        te = 0; #initial value of extraction tax for iteration
        tb_mat = [0, 1];  #initial value of  border adjustment and proportion of it (prop is mainly used for PC hybrid)

        ## calculate for optimal taxes by maximizing welfare
        tax_df=df.apply(minimization, axis=1, raw=False, args=(tb_mat, te, varphi, tax_scenario, ParaList))
        
        appended_df = pd.merge(df, tax_df, on=['region_scenario','regionbase'])

        ## back out other optimal values
        output_df = appended_df.apply(callback, axis=1, raw=False, result_type=None, args=(varphi, tax_scenario, ParaList))
        output_df = pd.merge(tax_df, output_df, on=['region_scenario','regionbase'])
        output.append(output_df)
        print(varphi)

    output = pd.concat(output, axis=0, join='outer',  ignore_index=False, keys=None, levels=None, names=None, verify_integrity=False,copy=True)
    output.reset_index(level=0, inplace=True)
    output = output.sort_values(by=['region_scenario','varphi'])
    if tax_scenario['tax_sce']=='purete' or tax_scenario['tax_sce']=='EP_hybrid':
        output.te[output.Qe_prime==0]=output.pe+output.tb

    print(tax_scenario['tax_sce'])
    return output


In [8]:
## apply the above for each tax scenario
output_all=tax_scenario.apply(cases, axis=1, args=(alpha, theta, sigma, sigmastar, epsilonD,epsilonDstar, epsilonS,epsilonSstar, beta, gamma, logit))

2
Baseline
2
Unilateral
2
purete
(array([1.]), {'nfev': 13, 'fjac': array([[-1.]]), 'r': array([nan]), 'qtf': array([-21.61870264]), 'fvec': array([21.61870264])}, 5, 'The iteration is not making good progress, as measured by the \n  improvement from the last ten iterations.')
(array([1.]), {'nfev': 13, 'fjac': array([[-1.]]), 'r': array([nan]), 'qtf': array([-21.61870264]), 'fvec': array([21.61870264])}, 5, 'The iteration is not making good progress, as measured by the \n  improvement from the last ten iterations.')
(array([1.]), {'nfev': 13, 'fjac': array([[-1.]]), 'r': array([nan]), 'qtf': array([-20.6506117]), 'fvec': array([20.6506117])}, 5, 'The iteration is not making good progress, as measured by the \n  improvement from the last ten iterations.')
(array([1.]), {'nfev': 13, 'fjac': array([[-1.]]), 'r': array([nan]), 'qtf': array([-20.6506117]), 'fvec': array([20.6506117])}, 5, 'The iteration is not making good progress, as measured by the \n  improvement from the last ten itera

In [12]:
output_list=[]
for i in range(1,len(tax_scenario)+1):
    output_list.append(output_all.loc[i])

In [13]:
Outcomes = pd.concat(output_list, axis=0, join='outer', ignore_index=False, keys=tax_scenario['tax_sce'], levels=None, verify_integrity=False,copy=True)
Outcomes.reset_index(level=0, inplace=True)

In [15]:
Outcomes

,tax_sce,region_scenario,tb,prop,te,varphi,pe,jxbar_prime,jmbar_prime,j0_prime,...,leakage3,chg_extraction,chg_production,chg_consumption,chg_Qeworld,pai_g,subsidy_ratio,welfare,welfare_noexternality,prop2
regionbase,,,,,,,,,,,,,,,,,,,,,
US as Home,Baseline,1,0.000000,1.000000,0.0,2.0,1.000000,0.015921,0.793579,0.015921,...,-9.515021,-5.081055e-07,9.964702e-07,1.998154e-06,2.100000e-07,1.027573e-07,0.000000,1.208037e-07,0.000001,NaN
EU28 as Home,Baseline,2,0.000000,1.000000,0.0,2.0,1.000000,0.017932,0.744319,0.017932,...,0.514918,-3.463258e-07,1.636902e-06,1.635722e-06,-3.176667e-06,7.480118e-08,0.000000,8.576907e-06,-0.000015,NaN
OECD37 as Home,Baseline,3,0.000000,1.000000,0.0,2.0,1.000000,0.049234,0.819498,0.049234,...,-0.102587,7.498559e-07,-1.172778e-06,-2.027811e-07,-1.976667e-06,-3.849407e-07,0.000000,5.297070e-07,-0.000004,NaN
World as Home,Baseline,4,0.000000,1.000000,0.0,2.0,1.000000,0.300000,0.999068,0.300000,...,-0.001836,1.030694e-09,-3.061387e-09,-3.061387e-09,-1.667333e-06,-4.122774e-09,0.000000,-1.547116e-06,-0.000003,NaN
China as Home,Baseline,5,0.000000,1.000000,0.0,2.0,1.000000,0.079652,0.920722,0.079652,...,0.762162,-8.742939e-07,6.043940e-07,7.164326e-07,-9.400000e-07,9.114455e-07,0.000000,1.870158e-05,0.000015,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
OECD37 as Home,EPC_hybrid,3,0.838402,0.998684,2.0,2.0,1.065062,0.040801,0.819446,0.035262,...,-0.168762,7.572569e-01,-2.107301e+00,-1.327858e+00,-7.868241e+00,-5.484505e+00,0.105608,8.188008e+00,-8.940192,0.507129
World as Home,EPC_hybrid,4,0.932487,0.983303,2.0,2.0,1.272133,0.258256,0.999064,0.235554,...,-0.001552,1.278888e-02,-2.792997e-02,-2.733338e-02,-1.760843e+01,-2.409239e-01,0.099060,9.074102e+00,-7.343504,0.435522
China as Home,EPC_hybrid,5,0.761847,0.992756,2.0,2.0,1.100714,0.067263,0.920592,0.059374,...,-0.414283,1.216606e+00,-2.064021e+00,-2.612524e+00,-6.306138e+00,-1.112924e+01,0.094056,1.129868e+01,-12.414732,0.486631


In [ ]:
if epsilonS == 0.5 and epsilonSstar == 0.5:
    Outcomes.to_csv('output/output_case{0}.csv'.format(case), header=True) 
elif epsilonS == 0.5 and epsilonSstar == 2:
    Outcomes.to_csv('output/output_case{0}_D_2.csv'.format(case), header=True) 
elif epsilonS == 2 and epsilonSstar == 0.5:
    Outcomes.to_csv('output/output_case{0}_2_D.csv'.format(case), header=True) 

In [21]:
!pip install palettable

     -------------------------------------- 111.8/111.8 kB 2.2 MB/s eta 0:00:00
